In [2]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
from rdflib import Graph

# === Globale Variablen ===
instance_file_path = None
output = widgets.Output()
result_graph = None

# === Auswahlfeld: Beispiel oder Upload ===
option_selector = widgets.ToggleButtons(
    options=[("Use example file", "example"), ("Upload your own", "upload")],
    description="Select input:",
    style={'description_width': 'initial'}
)

# === Upload-Widget (immer sichtbar) ===
upload_widget = widgets.FileUpload(
    accept='.ttl',
    multiple=False,
    description='Upload TTL file'
)

# === "Continue"-Button ===
continue_button = widgets.Button(description="Continue", button_style='primary')

# === Auswahlhandler ===
def on_option_change(change):
    output.clear_output()
    with output:
        if change['new'] == 'example':
            global instance_file_path
            instance_file_path = "Instance_Files/swemls-instances.ttl"
            print(f"📁 Example file selected:\n→ {instance_file_path}")
        elif change['new'] == 'upload':
            print("📤 Please upload a TTL file using the field below.")

option_selector.observe(on_option_change, names='value')

# === Upload-Handler ===
def on_upload(change):
    global instance_file_path
    if upload_widget.value:
        uploaded = upload_widget.value[0]
        file_name = uploaded['name']
        instance_file_path = file_name
        with open(file_name, 'wb') as f:
            f.write(uploaded['content'])
        output.clear_output()
        with output:
            print(f"✅ File uploaded and saved as:\n→ {file_name}")

upload_widget.observe(on_upload, names='value')

# === Continue-Button-Handler ===
def on_continue(b):
    global result_graph
    output.clear_output()
    if not instance_file_path:
        with output:
            print("⚠️ No file selected or uploaded.")
        return

    if not os.path.exists(instance_file_path):
        with output:
            print(f"❌ File not found on disk:\n→ {instance_file_path}")
        return

    # Versuche Datei mit rdflib zu laden
    g = Graph()
    try:
        g.parse(instance_file_path, format="turtle")
        result_graph = g
        with output:
            print(f"✅ RDF file successfully loaded!\n📄 Triples in graph: {len(g)}")
            print(f"🔗 Using file:\n→ {instance_file_path}")
    except Exception as e:
        with output:
            print(f"❌ Error parsing TTL file:\n→ {e}")

continue_button.on_click(on_continue)

# === Anzeige aller Elemente ===
display(option_selector, upload_widget, continue_button, output)


ModuleNotFoundError: No module named 'rdflib'